# Kryptologie LAB - Übung 10 - DSA

In [1]:
import math
import hashlib

## Parametergenerierung

In [2]:
# https://wiki.python.org/moin/BitManipulation
def bitLen(int_type):
    length = 0
    while (int_type):
        int_type >>= 1
        length += 1
    return(length)

In [3]:
# from Übung 07
import random

# x^m mod n
def potModN(x, m, n):
    y = 1
    b = m
    while b != 0:
        if (b & 0x1) == 1:
            y = (y * x) % n
        x = (x*x) % n
        b = b >> 1
    return y

class RSA:
    small_primes = [1,7,11,13,17, 19,23,29]

    def eea(self, a,b):
        k = 0
        r = [a, b]
        s = [1, 0]
        t = [0, 1]
        while r[k+1] != 0:
            k += 1
            q_k = r[k-1] // r[k]
            r.append(r[k-1]-q_k*r[k])
            s.append(s[k-1]-q_k*s[k])
            t.append(t[k-1]-q_k*t[k])
        return (r[k], s[k], t[k])

    def millerRabin(self, n):
        # determine k and m
        temp = n-1
        k = 0
        while (temp % 2) == 0:
            temp //= 2
            k += 1
        m = temp
        # determine a
        a = random.randrange(2,n)
        b = potModN(a, m, n)
        if b == 1:
            return True
        for i in range(1, k+1):
            if b == n-1:
                return True
            else:
                b = potModN(b, 2, n)
        return False
        

    def testPrim(self, n):
        success = False
        for i in range(8):
            success = self.millerRabin(n)
            if success: return True
        return False

    def genPrim(self, min_z = 1000, max_z = 10**130):
        z = random.randint(min_z, max_z)
        hit = False
        i = 1
        while not hit:
            number = 30 * z + self.small_primes[i]
            hit = self.testPrim(number)
            i += 1
            if i == 8:
                i = 0
                z += 1
        return number
    
    def genKeyPair(self):
        # generate primes
        p = self.genPrim()
        q = self.genPrim()
        while min([p,q]) * 1.2 > max([p,q]):
            q = self.genPrim()
        # gen e
        n = p * q
        phi_n = (p-1) * (q-1)
        hit = False
        while not hit:
            e = random.randint(1000, 10**180)
            ggt, d, _ = self.eea(e, phi_n)
            hit = ggt == 1
        if d < 0:
            d = phi_n + d
        return ((e,n), (d, n))


    def encrypt(self, x, public):
        e, n = public
        return self.potModN(x,e,n)

    def decrypt(self, x, private):
        d, n = private
        return self.potModN(x,d,n)

In [4]:
def ones(n):
    return (2**n)-1

def genQ(n):
    rsa = RSA()
    return rsa.genPrim(min_z=math.floor((1<<(n-1)) / 30), max_z=math.ceil(ones(n) / 30))

def genParameters(l: int, n: int):
    rsa = RSA()
    q = genQ(n)
    hit = False
    tries = 0
    while not hit:
        k = random.randint(2 ** (l - n), 2 ** (l - n + 1))
        p = q * k + 1
        hit = rsa.testPrim(p) and bitLen(p) == l
        tries += 1
        if tries > 100000:
            print("Generating new q")
            q = genQ(n)
            tries = 0
    print("q length: ", bitLen(q))
    print("p length: ", bitLen(p))

    g = 1
    while g == 1:
        h = random.randint(2, p-1)
        g = potModN(h,(p-1) // q, p)
    print("g: ", g)
    return p, q, g


## Schlüsselgenerierung

In [5]:
def genKey(p, q, g):
    x = random.randint(2,q)
    y = potModN(g,x,p)
    return x,y

## Signieren

In [6]:
hasher = hashlib.sha1()

def sha1(input):
    hasher.update(input)
    return hasher.digest()

# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [10]:
def sign(m,p,q,g,x):
    hashedMessage = int_from_bytes(sha1(m))
    r = 0
    s = 0
    rsa = RSA()
    while r == 0 or s == 0:
        j = random.randint(2, q)
        r = potModN(g,j,p) % q
        invertedJ, _, _ = rsa.eea(j,q)
        s = (invertedJ * (hashedMessage + r * x)) % q
    return r,s

def verify(m,p,q,g,y,r,s):
    hashedMessage = int_from_bytes(sha1(m))
    rsa = RSA()
    if 0 < r < q and 0 < s < q:
        w, _, _ = rsa.eea(s,q)
        u_1 = (hashedMessage * w) % q
        u_2 = (r*w) % q
        v = ((potModN(g,u_1,p) * potModN(y,u_2,p)) % p) % q
        if v == r:
            print("Valid signature")
            return True
        else:
            print("Invalid signature (v: ", v, " - r: ", r, ")")
            return False
    else:
        print("r and s in wrong intervalls")
        return False


In [8]:
p, q, g = genParameters(1024,160)
x,y = genKey(p,q,g)

q length:  160
p length:  1024
g:  13474087914347154663417055442739033242903728477996430796131965385857329503110282696550209961077850839761278592047677296102580458705731372653442498144130271709674617573163443365795678638738470759590679643182003048442382978943482293459197770626588530974997275188238174650631886711235899758131685811017177958578


In [11]:
message = b"Hallo, das ist ein Test"
r,s = sign(message,p,q,g,x)
print("r: ", bitLen(r))
print("s: ", bitLen(s))
verify(message,p,q,g,y,r,s)

r:  160
s:  159
Invalid signature (v:  919404015247550905792339837699358098697452453662  - r:  893841596410225988213505328988598947596880044625 )


False